Upcoming cell of code opens the camera and captures image and stores it in the directory

In [1]:
import cv2
import numpy as np
#Init camera
cap = cv2.VideoCapture(0)
#Face Detection using haarcascade File
face_cascade = cv2.CascadeClassifier('Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_alt.xml')
skip = 0
face_data = []
dataset_path = ('./CNN Face data/')
file_name = input("Enter the name of Person : ")
while True:
    ret,frame = cap.read()
    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    #The next line of code is written to only store the largest face in the window frame 
    faces = sorted(faces,key=lambda  f:f[2]*f[3])
    #start sorting from the last face since the last face is the largest in terms of area(w*h)
    for face in faces[-1:]:
        x,y,w,h = face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(123,200,69),2)
        #extract the required face or the region of the interest
        offset = 10   #Refers to adding an extra 10 pixels on all the sides of the required extracted face
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset] #By default face slicing is done in (y,x) manner
        face_section = cv2.resize(face_section,(100,100))
        skip +=1
        if skip%10==0 : #Store every 10th frame
            face_data.append(face_section)
            print(len(face_data)) #number of faces captured so far
    cv2.imshow("Video Frame",frame)
    cv2.imshow("Face section frame",face_section)
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
#Convert Our face list array into a numpy array
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0],-1))  #number of rows = number of faces
print(face_data.shape)
#Now we will save this data into file system
np.save(dataset_path+file_name+'.npy',face_data)
print("Data successfully save at :"+dataset_path+file_name+'.npy')    
cap.release()
cv2.destroyAllWindows()

Enter the name of Person : Sohum
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
(37, 30000)
Data successfully save at :./CNN Face data/Sohum.npy


In [2]:
import os

Reshape all the data into a (30,30000) matrix

In [3]:
from pathlib import Path
p = Path('./CNN Face data/')
dirs = p.glob("*")
labels = []
for folder_dir in dirs:
    label = str(folder_dir).split("\\")[-1]
    data = np.load('./CNN Face data/%s'%(label))
    data = data[:30:]
    lb = label[:-4]
    labels.append(lb)
    np.save('./CNN Face data/%s'%(label),data)

In [7]:
from keras.layers import *
from keras.models import Sequential
from keras.utils import np_utils
import tensorflow as tf
from tensorflow.python.client import device_lib
def CNN(train,test):
    
    Y_train = train[:,-1]
    X_train = train[:,:-1]
    x_train = X_train.reshape((-1,200,150,1))
    y_train = np_utils.to_categorical((Y_train))

    model = Sequential()
    model.add(Conv2D(128,(3,3),activation = 'relu',input_shape = (200,150,1)))
    model.add(Conv2D(256,(3,3),activation = 'relu'))
    model.add(Conv2D(512,(4,4),activation = 'relu'))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(128,(5,5),activation = 'relu'))
    model.add(Conv2D(64,(5,5),activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(len(labels),activation = 'softmax'))

    model.compile(optimizer = 'adadelta',loss = 'categorical_crossentropy',metrics = ["accuracy"])
    hist = model.fit(x_train,y_train,epochs=10,shuffle=True,validation_split=0.1,batch_size=16 )
    pred = model.predict_classes(x_test)
    return pred

In [8]:
cap = cv2.VideoCapture(0)
#Face Detection using haarcascade File
face_cascade = cv2.CascadeClassifier('Anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_alt.xml')
skip = 0
face_data = []     #Constitutes the X file of the training data
name_labels = []  #Constitutes the Y value of the training data
dataset_path = ('./CNN Face data/')
class_id = 0  #first file loaded in the video will have id zero
names = {} #MApping between the id and the name
### Data preperation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'): #open only the file having file extension as .npy
        #Maps the class id to the name of user
        #-4 is written so that the extension .npy is not added to dictionary along with the name
        names[class_id] = fx[:-4]
        data_item = np.load(dataset_path+fx) #append the facial data from the path to face_data  
        face_data.append(data_item)
        #Creating labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id +=1
        name_labels.append(target)
#Join the training data into one matrix
face_dataset = np.concatenate(face_data, axis = 0)
face_labels = np.concatenate(name_labels, axis = 0).reshape((-1,1))  #reshaping it gives the shape as (15,1) 
#15 is the number of images taken by the webcam
train_set = np.append(face_dataset,face_labels, axis = 1) #axis = 1 because we want to add another collumn to the dataset
##### TESTING PART #######
while True :
    ret, frame = cap.read()
    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    for face in faces:
        x,y,w,h = face
        #Get the region of interest
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        #predicted label and output
        out = CNN(train_set,face_section.flatten())  #we give the linear part of the face section
        #put name and rectangle around the name of user
        pred_name = names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
    cv2.imshow("Faces",frame)
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Train on 81 samples, validate on 9 samples
Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[16,512,193,143] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradients/max_pooling2d_1/MaxPool_grad/MaxPoolGrad (defined at C:\Users\gakha\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_1584]

Function call stack:
keras_scratch_graph
